#### **Models are trained on [JarvisLabs.ai](https://cloud.jarvislabs.ai) (a GPU cloud based platform offering modern and extremely easy to launch GPU instances) with a large batch size(~24). I will clean up and release the training code soon. Stay tuned **

# Refrences

https://www.kaggle.com/phalanx/train-swin-t-pytorch-lightning 
Folds used are the same that Abhishek thakur posted: https://www.kaggle.com/abhishek/same-old-creating-folds

fastai is used in training and inference code along with custom datasets, models and metric.

 ### Adding Modules

In [ ]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master') ## Importing Timm Library

In [ ]:
import os


import fastai 
import os
import warnings
from pprint import pprint
from glob import glob
from tqdm import tqdm
from fastai.vision.all import *
import torch

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from timm import create_model

#Importing Dataloaders

from sklearn.model_selection import StratifiedKFold
from torchvision.io import read_image
from torch.utils.data import DataLoader, Dataset

#Augmentation & Preprocessing Modules  

import albumentations
from albumentations.pytorch import ToTensorV2 
from albumentations import ImageOnlyTransform
import random
import timm
import cv2

#For enabling fp16

from torch.cuda.amp import autocast, GradScaler  

import warnings 
warnings.filterwarnings('ignore')    ##ignore warning

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') ### Enabling use of cuda

In [ ]:
class Config: 
    model_name='swin_base_patch4_window12_384'  # which model to use
    pretrained = False # enable pretrained weights
    train_dir = '../input/petfinder-pawpularity-score/train' # Train Image Directory
    train_csv = '../input/abhi-folds-petfinder/train_10folds.csv' # Train Csv Location
    
    test_dir = '../input/petfinder-pawpularity-score/test' # Test Image Directory
    test_csv = '../input/petfinder-pawpularity-score/test.csv' # Test Csv Location
    sample_csv = '../input/petfinder-pawpularity-score/sample_submission.csv' # Sample Submission
    
    image_size= 384   # Image Size for training / inference
    num_workers=4  #number of worker
    targets = 1  # number of targets
    batch_size=6 # Batch size
    weight_decay=1e-4  #Weight decay for optimizer 
    seed=42 # SEED
    n_fold=10  #No of folds
    trn_fold=[0, 1, 2, 3,4,5,6,7,8,9]  #Folds to be used in training / inference
    train=False  
    model_dir = '../input/petfinder-v0-models/exp0'  # Saved models directory

In [ ]:
def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_torch(seed=Config.seed)

## Defining Dataset

In [ ]:
class PetDataset:
    def __init__(self, df,image_path, augmentations):
        self.image_path = image_path
        self.df = df
        self.augmentations = augmentations
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, item):
        id_ = self.df.Id.iloc[item] 
        path = f'{self.image_path}/{id_}.jpg' # Getting Image paths for samples

        targets = self.df.Pawpularity.iloc[item] # Target value for samples
        image = cv2.imread(path)  # Reading Image
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Converting BGR to RGB images
        
        if self.augmentations is not None:
            augmented = self.augmentations(image=image)  # Apply Augmentations
            image = augmented["image"]
            
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)    
        image = torch.tensor(image, dtype=torch.float) 
        
        targets = torch.tensor(targets, dtype=torch.float) / 100  # For classification
        return image, targets
            

## Augmentations

In [ ]:
def get_transforms(*, data):

    if data == 'valid':
        return albumentations.Compose([
            albumentations.Resize(Config.image_size, Config.image_size),   # Resize all images to 384 image size.
            albumentations.Normalize(     
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
            p=1.0,
        ),           

        ])



Defining Regression loss

In [ ]:
class RMSELoss(nn.Module):
    def __init__(self, eps=1e-6):
        super().__init__()
        self.mse = nn.MSELoss()
        self.eps = eps

    def forward(self, yhat, y):
        loss = torch.sqrt(self.mse(yhat, y) + self.eps)
        return loss

In [ ]:
# Cell
import gc
def mse(inp,targ):
    "Mean squared error between `inp` and `targ`."
    return F.mse_loss(*flatten_check(inp,targ))

# Cell

def _rmse(inp, targ): return 100*torch.sqrt(F.mse_loss(inp, targ))
rmse = AccumMetric(_rmse)
rmse.__doc__ = "rrr"

In [ ]:
def petfinder_rmse(input,target):
    return 100*torch.sqrt(F.mse_loss(F.sigmoid(input.flatten()), target))

## Model Class

In [ ]:
class PetModel(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.backbone = timm.create_model(self.config.model_name, pretrained=self.config.pretrained)
        self.n_features = self.backbone.head.in_features
        self.backbone.head = nn.Identity()
        self.fc = nn.Linear(self.n_features, self.config.targets)
        
    def forward(self, image):
#         print('done')
        feature = self.backbone(image)
        output = self.fc(feature)


        return output

## Generating Predictions

In [ ]:
def run(Config):
    for fold_num in Config.trn_fold:    
        print('*****************************************')
        print(f'Training Fold {fold_num}')
        print('*****************************************')
        bs = Config.batch_size


        kernel_type = Config.model_dir
        df = pd.read_csv(Config.train_csv)[['Id', 'Pawpularity', 'kfold']]
        df['is_valid'] = df.kfold.apply(lambda x: x==fold_num)
        
        validation_fold = df.query('is_valid==True').reset_index(drop=True, inplace=False)
        valid_ds = PetDataset(validation_fold,Config.train_dir,augmentations = get_transforms(data='valid'))
        valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=bs, num_workers=4,pin_memory=True)

        dls = valid_dl
        print(f'- Validation Samples : {len(valid_ds)}')

        print(pd.read_csv(f'../input/petfinder-v0-models/exp0/{fold_num}logs.csv'))

        model = PetModel(Config)
        learn = Learner(dls, model, loss_func=BCEWithLogitsLossFlat(), metrics=[petfinder_rmse])#.to_fp16()

        model_path = f'{kernel_type}/fold_{fold_num}.pth'  # trained models path
        learn.model.load_state_dict(torch.load(model_path)['model']) # Loading trained model weights
        learn.model.eval()
        learn.model.cuda()

        

        
        test_df = pd.read_csv(Config.test_csv)   
        test_df['Pawpularity'] = 0
        test_ds = PetDataset(test_df,Config.test_dir,augmentations = get_transforms(data='valid'))  
        inds = list(test_df.index)
        test_dl = torch.utils.data.DataLoader(test_ds, batch_size=bs, num_workers=4, shuffle=False,pin_memory=True) #test loader

        preds = []
        for xb, _ in progress_bar(test_dl):
            with torch.no_grad(): 
                output = learn.model(xb.cuda())    # Making predictions
            preds.append(torch.sigmoid(output.float()).squeeze().cpu()) # storing predictions in list
        preds = torch.cat(preds).numpy().flatten()
        
        if fold_num == 0:
            sample_df = pd.read_csv(Config.sample_csv)
            sample_df['Pawpularity'] = 0
        sample_df.loc[inds,'Pawpularity'] += 100 * preds / len(Config.trn_fold)  # ensemble multiple folds predictions
        
        del df,model,learn,test_ds,test_dl,valid_dl,preds
        gc.collect()
        
    return sample_df

        


In [ ]:
sub = run(Config)

In [ ]:
sub.to_csv('submission.csv',index=False)

In [ ]:
sub